<div align="center">
    <span style="font-size:30px">
        <strong>
            <!-- Símbolo de Python -->
            <img
                src="https://cdn3.emoji.gg/emojis/1887_python.png"
                style="margin-bottom:-5px"
                width="30px" 
                height="30px"
            >
            <!-- Título -->
            Python para Geólogos
            <!-- Versión -->
            <img 
                src="https://img.shields.io/github/release/kevinalexandr19/manual-python-geologia.svg?style=flat&label=&color=blue"
                style="margin-bottom:-2px" 
                width="40px"
            >
        </strong>
    </span>
    <br>
    <span>
        <!-- Github del proyecto -->
        <a href="https://github.com/kevinalexandr19/manual-python-geologia" target="_blank">
            <img src="https://img.shields.io/github/stars/kevinalexandr19/manual-python-geologia.svg?style=social&label=Github Repo">
        </a>
        &nbsp;&nbsp;
        <!-- Licencia -->
        <img src="https://img.shields.io/github/license/kevinalexandr19/manual-python-geologia.svg?color=forestgreen">
        &nbsp;&nbsp;
        <!-- Release date -->
        <img src="https://img.shields.io/github/release-date/kevinalexandr19/manual-python-geologia?color=gold">
    </span>
    <br>
    <span>
        <!-- Perfil de LinkedIn -->
        <a target="_blank" href="https://www.linkedin.com/in/kevin-alexander-gomez/">
            <img src="https://img.shields.io/badge/-Kevin Alexander Gomez-5eba00?style=social&logo=linkedin">
        </a>
        &nbsp;&nbsp;
        <!-- Perfil de Github -->
        <a target="_blank" href="https://github.com/kevinalexandr19">
            <img src="https://img.shields.io/github/followers/kevinalexandr19.svg?style=social&label=kevinalexandr19&maxAge=2592000">
        </a>
    </span>
    <br>
</div>

***

<span style="color:lightgreen; font-size:25px">**PG301 - Geoestadística** </span>

Bienvenido al curso!!!

Vamos a revisar las bases de la <span style="color:gold">geoestadística</span> usando ejemplos en Python. <br>
Es necesario que tengas un conocimiento previo en programación con Python, estadística y geología general.

<span style="color:gold; font-size:20px">**Declustering** </span>

***
- [Representatividad en el muestreo geoestadístico](#parte-1)
- [¿Qué es declustering?](#parte-2)
- [Declustering con Python](#parte-3)

***

<a id="parte-1"></a>

### <span style="color:lightgreen">**Representatividad en el muestreo geoestadístico**</span>
***
Se dice que una muestra es <span style="color:gold">**representativa**</span> cuando refleja las características esenciales de la población de la cuál fue extraída.

En general, debemos asumir que todas las muestras tomadas del campo se encuentran sesgadas de alguna forma.

Si tuviéramos que realizar un muestreo tomando en cuenta la representatividad de las muestras, teóricamente, tendríamos dos opciones:

- Realizar un **muestreo aleatorio**, en donde asumimos que cada elemento de la población tiene la misma probabilidad de ser extraída.
- Realizar un **muestreo sistemático**, en donde las muestras son extraídas a intervalos regulares (igualmente espaciadas).
***

<a id="parte-2"></a>

### <span style="color:lightgreen">**¿Qué es declustering?**</span>
***

Durante el muestreo, es frecuente encontrar áreas con una mayor concentración de muestras. Esta práctica puede llevar a un sesgo en la estadística general de los datos, ya que <span style="color:#43c6ac">la distribución irregular de las muestras reduce la representatividad del volumen de interés</span>.

Para tratar este sesgo en la toma de muestras, se puede utilizar el <span style="color:gold">declustering</span> o <span style="color:gold">desagrupamiento</span>. Esta técnica consiste en asignar un peso a las muestras basándose en su proximidad a las muestras circundantes. Las ponderaciones son mayores a 0 y, en total, suman 1.

Para evaluar la proximidad, se utiliza una malla que divide el área en celdas de un tamaño específico. Cada celda puede contener varias o ninguna de las muestras; cuantas más muestras tenga una celda, menor será la ponderación asignada. De la misma forma, una muestra alejada de las demás tendrá una ponderación más alta que aquellas que se encuentren agrupadas.

<center>
    <img src="resources/declustering_weights.png" width="600"/>
</center>

<br>

Si el tamaño de la celda fuera equivalente al tamaño de la malla, el promedio de los datos sería equivalente al promedio sin desagrupar. Por otro lado, si el tamaño de la celda fuera extremadamente pequeño, el promedio de los datos también sería equivalente al promedio sin desagrupar. Por lo tanto, <span style="color:#43c6ac">existe un tamaño de celda óptimo entre estos extremos que se debe usar para desagrupar los datos</span>.

Además, debemos tener en cuenta que la ubicación de la malla también influye en la ponderación individual de cada muestra. Para resolver este problema, se pueden tomar varias ubicaciones aleatorias y promediar las ponderaciones individuales asignadas a cada muestra.

Una vez asignadas las ponderaciones de desagrupamiento a cada muestra, se pueden obtener medidas estadísticas desagrupadas, como el promedio, la varianza, la covarianza, etc.

***

<a id="parte-3"></a>

### <span style="color:lightgreen">**Declustering con Python**</span>
***
Empezaremos importando las librerías que utilizaremos en este tutorial:

In [ ]:
# Librería geoestadística
import geostatspy.GSLIB as GSLIB          # GSLIB: herramientas, visualizador y wrapper
import geostatspy.geostats as geostats    # Métodos de GSLIB convertidos a Python

# Librerías fundamentales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Funciones estadísticas
from statsmodels.stats.weightstats import DescrStatsW

Y abriremos el archivo `data_sesgada.csv`, que contiene la información a desagrupar.

In [ ]:
data = pd.read_csv("files/data_sesgada.csv")

In [ ]:
data.head()

Observamos que `data` tiene las siguientes columnas:
- `X`, `Y`: coordenadas
- `Facies`: 1 para arenisca y 0 para intercalaciones de arenisca y lutita
- `Porosidad`: porosidad en fracción (%)
- `Permeabilidad` : permeabilidad en miliDarcy (mD)

In [ ]:
# Resumen estadístico
data.describe()

Ahora, vamos a especificar el **área de interés**.

Es común delimitar manualmente el rango de las coordenadas X e Y. También estableceremos un rango para la columna de `Porosidad` y un mapa de colores para la visualización.

In [ ]:
# Coordenadas
xmin, xmax = 0., 1000.
ymin, ymax = 0., 1000.

# Porosidad
pormin, pormax = 0.05, 0.25

# Mapa de colores
cmap = plt.cm.inferno

Para mostrar el área de interés en un gráfico, crearemos una figura similar al `locmap` de GSLIB:

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(7, 6), subplot_kw={"aspect": 1})

# Diagrama de dispersión
im = ax.scatter(data=data, x="X", y="Y", c="Porosidad", cmap=cmap, edgecolor="black", alpha=0.8)
im.set_clim(pormin, pormax)

# Barra de colores
cbar = fig.colorbar(im, ax=ax)
cbar.set_label("Porosidad (%)", rotation=270, labelpad=25)

# Límites
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

# Texto
ax.set_title("Data - Porosidad")
ax.set_xlabel("X (m)")
ax.set_ylabel("Y (m)")

# Grilla
ax.grid(lw=0.5, alpha=0.5, c="k")
ax.set_axisbelow(True)

plt.show()

Podemos observar que en las regiones de alta porosidad hay un mayor número de muestras. Esto se puede considerar como un <span style="color:gold">muestreo preferencial</span> o <span style="color:gold">selectivo</span>.

Debido a este sesgo, no podemos utilizar la estadística general para representar adecuadamente esta región. Es necesario realizar una corrección por el agrupamiento de las muestras en las áreas de alta porosidad.

En este caso, utilizaremos el desagrupamiento por celdas y buscaremos minimizar la media desagrupada. Visualmente, podemos notar que un tamaño de celda adecuado debería estar entre 100 y 200 metros.

Para realizar el desagrupamiento, utilizaremos la función `declus` reimplementada de GSLIB en Python, a través del módulo `geostats`.

In [ ]:
# Detalles de la función
geostats.declus

Observamos que la función `declus` tiene los siguientes parámetros:

- `df`: el DataFrame con la información
- `xcol`, `ycol`: las columnas de coordenadas x e y
- `vcol`: la columna que contiene la variable de interés
- `iminmax`: puede ser `0`/`False` si se usa un tamaño de celda que maximice la media desagrupada o `1`/`True` si se usa un tamaño que minimice la media desagrupada.
- `noff`: número de ubicaciones aleatorias para la malla
- `ncell`: número de tamaños de celda a probar por cada malla
- `cmin`: tamaño mínimo de celda
- `cmax`: tamaño máximo de celda

Probaremos con un amplio rango de tamaño de celdas, de 10 m a 2000 m, y eligiremos aquel tamaño que minimice la media desagrupada. También usaremos 10 ubicaciones aleatorias de malla y 100 tamaños de celda a probar por cada malla.

In [ ]:
wts, cell_sizes, dmeans = geostats.declus(df=data, xcol="X", ycol="Y", vcol="Porosidad",
                                          iminmax=1, noff=10, ncell=100, cmin=10, cmax=2000)

El resultado de la función `declus` está compuesto por:

- `wts`: un arreglo que contiene las ponderaciones desagrupadas de cada dato (la suma es equivalente al número de datos, el valor de 1 indica un peso nominal)
- `cell_sizes`: un arreglo con los tamaños de celda considerados
- `dmeans`: un arreglo con las medias desagrupadas, calculadas por cada tamaño de celda en `cell_sizes`

Ahora, usaremos la función para obtener las ponderaciones y generar un gráfico para elegir el tamaño de celda óptimo.

In [ ]:
# Creamos una nueva columna con las ponderaciones
data["wts"] = wts
data.head()

Y ahora graficaremos la distribución de las ponderaciones sobre el área de interés:

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(7, 6), subplot_kw={"aspect": 1})

# Diagrama de dispersión
im = ax.scatter(data=data, x="X", y="Y", c="wts", cmap=cmap, edgecolor="black", alpha=0.8)
im.set_clim(0.25, 4) # Rango de valores de ponderación

# Barra de colores
cbar = fig.colorbar(im, ax=ax)
cbar.set_label("Ponderaciones", rotation=270, labelpad=25)

# Límites
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

# Texto
ax.set_title("Data - Ponderaciones")
ax.set_xlabel("X (m)")
ax.set_ylabel("Y (m)")

# Grilla
ax.grid(lw=0.5, alpha=0.5, c="k")
ax.set_axisbelow(True)

plt.show()

Observamos que las ponderaciones varían de acuerdo a la densidad de muestras en la región, por lo tanto, hemos conseguido desagrupar las muestras.

Ahora, crearemos una figura resumen en la cual graficaremos lo siguiente:

- El área de interés con las ponderaciones asignadas,
- Un histograma mostrando la distribución de ponderaciones,
- Una comparación entre las distribuciones de porosidad para las muestras sin desagrupar y desagrupadas.

In [ ]:
# Figura principal
fig, axs = plt.subplots(2, 2, figsize=(10, 7))

# Figura 1. Área de interés con ponderaciones
im = axs[0, 0].scatter(data=data, x="X", y="Y", c="wts", s=40, cmap=cmap, edgecolor="black", alpha=0.8)
im.set_clim(0.25, 4)

# Barra de colores
cbar = fig.colorbar(im, ax=axs[0, 0])
cbar.set_label("Ponderaciones", rotation=270, labelpad=25)

# Límites
axs[0, 0].set_xlim(xmin, xmax)
axs[0, 0].set_ylim(ymin, ymax)
axs[0, 0].set(aspect=1)

# Texto
axs[0, 0].set_title("Data - Ponderaciones")
axs[0, 0].set_xlabel("X (m)")
axs[0, 0].set_ylabel("Y (m)")

# Figura 2. Histograma de ponderaciones
axs[0, 1].hist(data=data, x="wts", bins=20, color="darkorange", edgecolor="black")
axs[0, 1].margins(x=0)

# Título y nombres
axs[0, 1].set_title("Ponderaciones de desagrupamiento")
axs[0, 1].set_xlabel("Ponderaciones")
axs[0, 1].set_ylabel("Frecuencia")

# Figura 3. Porosidad sin desagrupar
axs[1, 0].hist(data=data, x="Porosidad", bins=20, color="darkorange", edgecolor="black")
axs[1, 0].set_xlim(pormin, pormax)

# Texto
axs[1, 0].set_title("Porosidad sin desagrupar")

# Figura 4. Porosidad desagrupada
axs[1, 1].hist(data=data, x="Porosidad", weights="wts", bins=20, color="darkorange", edgecolor="black")
axs[1, 1].set_xlim(pormin, pormax)

# Texto
axs[1, 1].set_title("Porosidad desagrupada")

# Detalles adicionales
axs[1, 0].sharey(axs[1, 1]) # Compartir el eje Y entre axs[1, 0] y axs[1, 1]

# Texto para ambas figuras
for ax in axs[1]:
    ax.set_xlabel("Porosidad (%)")
    ax.set_ylabel("Frecuencia")

# Grilla
for ax in axs.flatten():
    ax.grid(lw=0.5, alpha=0.5, c="k")
    ax.set_axisbelow(True)

plt.tight_layout()

También mostraremos un resumen de la variación en la media de porosidad al desagrupar los datos:

In [ ]:
mean = np.average(data["Porosidad"].values)
dmean = np.average(data["Porosidad"].values, weights=data["wts"].values)
correction = (mean - dmean) / mean

print(f"La media de porosidad sin desagrupar es de {mean:.3f}")
print(f"La media de porosidad desagrupada es de {dmean:.3f}")
print(f"Corrección de {correction:.2%}")

Ahora, crearemos un gráfico mostrando la **media desagrupada de porosidad** vs. el **tamaño de celda de desagrupamiento** a través de las 100 repeticiones que se realizaron.\
Recordemos que cuando el tamaño de celda es demasiado grande o demasiado pequeño, la media desagrupada es equivalente a la media sin desagrupar.

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(8, 6))

# Diagrama de dispersión
ax.scatter(cell_sizes, dmeans, s=15, alpha=0.8, edgecolor="black", facecolor="darkorange")

# Ticks del eje x
ax.set_xticks(np.linspace(0, 2000, 11))

# Límites de la figura
ax.margins(x=0)
ax.set_ylim(0.10, 0.16)

# Título y nombres
ax.set_title("Media desagrupada de Porosidad vs. Tamaño de celda")
ax.set_xlabel("Tamaño de celda (m)")
ax.set_ylabel("Media desagrupada de Porosidad (%)")

plt.show()

Notamos que el tamaño de celda óptimo se encuentra aproxidamente en 200 metros. Graficaremos unas líneas adicionales en la figura:

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(8, 6))

# Diagrama de dispersión
ax.scatter(cell_sizes, dmeans, s=15, alpha=0.8, edgecolor="black", facecolor="darkorange")

# Ticks del eje x
ax.set_xticks(np.linspace(0, 2000, 11))

# Límites de la figura
ax.margins(x=0)
ax.set_ylim(0.10, 0.16)

# Título y nombres
ax.set_title("Media desagrupada de Porosidad vs. Tamaño de celda")
ax.set_xlabel("Tamaño de celda (m)")
ax.set_ylabel("Media desagrupada de Porosidad (%)")

# Tamaño de celda óptimo
ax.plot([0, 2000], [mean, mean], c="black")
ax.plot([200, 200], [0.10, 0.16], c="black", ls="dashed")

# Texto en la figura
ax.text(300, 0.136, "Media sin desagrupar de Porosidad")
ax.text(230, 0.151, "Tamaño de\ncelda óptimo")

plt.show()

Finalizaremos realizando una estadística descriptiva de los datos desagrupados.

Si bien podemos calcular la media, varianza y desviación estándar manualmente, también podemos utilizar la función `DescrStatsW` del módulo `statsmodels.stats.weights`. Esta función nos permite agregar ponderaciones a un conjunto de datos a través de los siguientes parámetros:

- `data`: es el arreglo que contiene los datos
- `weights`: son las ponderaciones a utilizar para cada dato

Asignaremos el conjunto ponderado a una variable llamada `ddata`:

In [ ]:
ddata = DescrStatsW(data=data["Porosidad"].values, weights=data["wts"])

Y por último, generaremos un resumen estadístico de los datos sin desagrupar y desagrupados de Porosidad:

In [ ]:
# Cálculo manual de los datos sin desagrupar
mean = np.average(data["Porosidad"].values)
var = np.var(data["Porosidad"].values)
std = np.std(data["Porosidad"].values)

# Resumen estadístico
print(f"Estadística sin desagrupar - Porosidad")
print(f"    Media: {mean:.3f}")
print(f"    Varianza: {var:.5f}")
print(f"    Desviación estándar: {std:.3f}\n")

print(f"Estadística desagrupada - Porosidad")
print(f"    Media: {ddata.mean:.3f}")
print(f"    Varianza: {ddata.var:.5f}")
print(f"    Desviación estándar: {ddata.std:.3f}")

En conclusión, realizar un desagrupamiento de los datos nos permite corregir el sesgo de muestreo.

***